In [5]:
from bs4 import BeautifulSoup
import bs4
from playwright.async_api import async_playwright
import requests

In [18]:
url = 'https://assist.org/transfer/results?year=75&institution=79&agreement=113&agreementType=from&viewAgreementsOptions=true&view=agreement&viewBy=major&viewSendingAgreements=false&viewByKey=75%2F113%2Fto%2F79%2FMajor%2F607b828c-8ba3-411b-7de1-08dcb87d5deb'

In [19]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=True)
page = await browser.new_page()
await page.goto(url, wait_until='networkidle')
await page.wait_for_selector('.articRow')

<JSHandle preview=JSHandle@node>

In [20]:
content = await page.content()
soup = BeautifulSoup(content, 'lxml')
articRows = soup.find_all('div', class_='articRow')
print(f"Found {len(articRows)} receiving course blocks.")
type(articRows[0])

Found 23 receiving course blocks.


bs4.element.Tag

In [53]:
example_articRow = articRows[5] # simplest case one one course for both receiving and sending 5 is one and

In [ ]:
def clean_units(units: str) -> float:
    """
    ### Description:
        Remove the 'unit' suffix and convert to float.
    ### Args:
        units (str): The units string to clean, e.g., "4.00units" or "3.0 unit".
    ### Returns:
        float: The cleaned units as a float, e.g., 4.0 or 3.0.
    ### Raises:
        ValueError: If the units string is not in a valid format.
    """
    removed_suffix = units.replace('units', '').replace('unit', '').strip()
    try:
        return float(removed_suffix)
    except ValueError as e:
        print(f"Error converting units to float: {e}")
        raise ValueError(f"Invalid units format: {units}")

def clean_full_course(course: str) -> dict[str, str]:
    """
    ### Description:
        Clean the course string to extract the subject and number.
    ### Args:
        course (str): The full course string, e.g., "CS 101".
    ### Returns:
        dict: A dictionary with 'subject' and 'number' keys.
    """
    parts = course.split()
    if len(parts) < 2:
        raise ValueError(f"Invalid course format: {course}")
    return {
        'subject': parts[0],
        'number': ' '.join(parts[1:])
    }
    
def process_course_line(courseLine: bs4.element.Tag):
    try:
        course = courseLine.find('div', class_='prefixCourseNumber').text.strip() # type: ignore
        title = courseLine.find('div', class_='courseTitle').text.strip() # type: ignore
        units = courseLine.find('div', class_='courseUnits').text.strip() # type: ignore
        course_subject = clean_full_course(course)["subject"]
        course_number = clean_full_course(course)["number"]
        return {
            'type': 'course',
            'subject': course_subject,
            'number': course_number,
            'title': title,
            'units': clean_units(units)
        }
    except AttributeError as e:
        print(f"Error processing course line: {e}")
        return None

In [ ]:
def handle_bracket(bracket: bs4.element.Tag):
    """
    ### Description:
        Handle the bracket element and extract its text.
    ### Args:
        bracket (bs4.element.Tag): The bracket element to process.
    ### Returns:
        str: The cleaned text from the bracket.
    """
    assert isinstance(bracket, bs4.element.Tag), "Expected a BeautifulSoup Tag"
    assert bracket.name == 'div', "Expected a div element"
    assert 'bracketWrapper' in bracket.get('class', []), "Expected a bracket class" #type: ignore : again [] saves us
        
    bracket_content = bracket.find('div', class_='bracketContent')
    
    # make sure we only have ands
    cojunctions = bracket_content.find_all('awc-view-conjunction')
    if len(cojunctions) == 0:
        raise ValueError("No conjunctions found in bracket content")
    for conjunction in cojunctions:
        if conjunction.text.strip() != 'And':
            raise ValueError(f"Unexpected conjunction: {conjunction.text.strip()} in bracket")
    
    course_lines = bracket_content.find_all('div', class_='courseLine')
    if not course_lines:
        raise ValueError("No course lines found in bracket content")
    courses = []
    for course_line in course_lines:
        course = process_course_line(course_line)
        if course:
            courses.append(course)
    
    return courses

In [ ]:
def handle_main_block(mainBlock: bs4.element.Tag):
    """
    ### Description:
        Handle the main block element and extract its text.
    ### Args:
        mainBlock (bs4.element.Tag): The main block element to process.
    ### Returns:
        dict: A dictionary with the type and text of the main block.
    """
    assert isinstance(mainBlock, bs4.element.Tag), "Expected a BeautifulSoup Tag"
    assert mainBlock.name == 'div', "Expected a div element"
    assert mainBlock.find(class_="courseLine") is not None, "Expected a courseLine class in mainBlock" # type: ignore
    
    if (mainBlock.find('div', class_='bracketWrapper') is not None):
        # handle bracket
        bracket = mainBlock.find('div', class_='bracketWrapper')
        courses_in_bracket = handle_bracket(bracket)
        return {
            "type": "CourseGroup",
            "courseConjunction": "And",
            "courses": courses_in_bracket
        }
    else:
        # for now only support one courseLine
        courseLine = mainBlock.find('div', class_='courseLine')
        if courseLine:
            course_data = process_course_line(courseLine) # type: ignore
            return course_data
        else:
            raise ValueError("No courseLine found.")
    

In [73]:
# handle a rowReceiving
def handle_rowReceiving(rowReceiving: bs4.element.Tag):
    assert rowReceiving.name == 'div' and 'rowReceiving' in rowReceiving.get('class', []), f"Expected a 'div' with class 'rowReceiving', got {rowReceiving.name} with classes {rowReceiving.get('class', [])}" # type: ignore
    try:
        return handle_main_block(rowReceiving) 
    except ValueError as e:
        if str(e) == "No courseLine found.":
            raise ValueError("No courseLine found in rowReceiving.")
        else:
            raise e

In [74]:
def handle_rowSending(rowSending: bs4.element.Tag):
    assert rowSending.name == 'div' and 'rowSending' in rowSending.get('class', []), f"Expected a 'div' with class 'rowSending', got {rowSending.name} with classes {rowSending.get('class', [])}" # type: ignore
    
    mainContent = rowSending.find('div', class_='view_sending__content')
    if not mainContent:
        raise ValueError("No main content found in rowSending.")
    try:
        return handle_main_block(mainContent)
    except ValueError as e:
        if str(e) == "No courseLine found.":
            raise ValueError("No courseLine found in rowSending.")
        else:
            raise e 

In [49]:
def process_artic_row(row: bs4.element.Tag):
    try:
        receiving_html = row.find('div', class_='rowReceiving')  # type: ignore
        receiving_course = handle_rowReceiving(receiving_html) # type: ignore
        sending_html = row.find('div', class_='rowSending')  # type: ignore
        sending_course = handle_rowSending(sending_html) # type: ignore
    except Exception as e:
        print(f"Error processing receiving course: {e}")
        raise ValueError("Failed to process articRow.")
    
    return {
        "receiving": receiving_course,
        "sending": sending_course
    }

In [75]:
process_artic_row(example_articRow)

Processing main block without bracket...
Processing bracket in main block...
Processing bracket...


{'receiving': {'type': 'course',
  'subject': 'PHYSICS',
  'number': '7B',
  'title': 'Physics for Scientists and Engineers',
  'units': 4.0},
 'sending': {'type': 'CourseGroup',
  'courseConjunction': 'And',
  'courses': [{'type': 'course',
    'subject': 'PHYS',
    'number': '4B',
    'title': 'Physics for Scientists and Engineers: Electricity and Magnetism',
    'units': 6.0},
   {'type': 'course',
    'subject': 'PHYS',
    'number': '4C',
    'title': 'Physics for Scientists and Engineers: Fluids, Waves, Optics and Thermodynamics',
    'units': 6.0}]}}